## Displaying Mermaid Diagrams in a Jupyter Notebook Using Python

This code is based upon code at <https://mermaid.js.org/config/Tutorials.html#jupyter-integration-with-mermaid-js>.

In [ ]:
import base64
import re
from IPython.display import Image, display
from pathlib import Path


def mm_ink(graphbytes):
    """Return a mermaid.ink URL for the provided bytes."""
    return "https://mermaid.ink/img/" + base64.b64encode(graphbytes).decode("ascii")


def mm_display(graphbytes):
    """Display the mermaid image for the provided bytes."""
    display(Image(url=mm_ink(graphbytes)))


def _strip_wrappers(text):
    """Strip common wrappers so a file can contain a Mermaid diagram in several forms.

    This function supports files containing:
    - a wrapper call around the diagram (for example, mm(...))
    - a fenced code block (triple-backtick block containing the diagram)
    - plain Mermaid source
    """
    if not text:
        return ""

    # Remove leading/trailing wrapper like mm("""...""") or mm('''...''')
    text = re.sub(r"^\s*mm\(\s*(?:\"\"\"|\'\'\')", "", text, flags=re.I)
    text = re.sub(r"(?:\"\"\"|\'\'\')\s*\)\s*;?\s*$", "", text, flags=re.I|re.S)

    # If file uses a fenced code block, extract inner content
    lines = text.splitlines()
    if lines and lines[0].strip().startswith("```"):
        # find the last fence line
        end_idx = None
        for i in range(len(lines)-1, -1, -1):
            if lines[i].strip().startswith("```") and i != 0:
                end_idx = i
                break
        if end_idx is not None:
            return "\n".join(lines[1:end_idx]).strip()
        # fallback: return everything except the first line
        return "\n".join(lines[1:]).strip()

    return text.strip()


def mm(graph):
    """Display a mermaid graph (string)."""
    graphbytes = graph.encode('ascii', errors='replace')
    mm_display(graphbytes)


def mm_link(graph):
    """Return a mermaid.ink link for the graph string."""
    graphbytes = graph.encode('ascii', errors='replace')
    return mm_ink(graphbytes)


def mm_path(path):
    """Read a file (path can be str or Path), strip wrappers and display the mermaid diagram."""
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"{p} not found")
    raw = p.read_text(encoding='utf-8')
    mermaid = _strip_wrappers(raw)
    if not mermaid:
        raise ValueError(f"No mermaid content found in {p}")
    graphbytes = mermaid.encode('ascii', errors='replace')
    mm_display(graphbytes)

# Example usage (run this cell first to define helpers):
# mm_path(r"C:\Users\easts\Desktop\digital-mgt\flowchartLR-for-juptyer.mmd")
# mm_path(r"C:\Users\easts\Desktop\digital-mgt\flowchart LR.md")


In [ ]:
import requests
import os
import base64
from IPython.display import SVG, display
from pathlib import Path

def mm_svg(graph_or_path, is_path=False):
    """
    Create and display an SVG version of a Mermaid diagram.

    Args:
        graph_or_path: Either a string containing Mermaid syntax or a path to a file
        is_path: Boolean indicating if graph_or_path is a file path

    Returns:
        str: The URL used to fetch the SVG (GET) or a short indicator if POST fallback used.
    """
    # Get Mermaid content, either from string or file
    if is_path:
        p = Path(graph_or_path)
        if not p.exists():
            raise FileNotFoundError(f"{p} not found")
        raw = p.read_text(encoding='utf-8')
        mermaid = _strip_wrappers(raw)
    else:
        mermaid = _strip_wrappers(graph_or_path)

    if not mermaid:
        raise ValueError("No mermaid content provided")

    # Encode using URL-safe base64 and strip padding to make a compact URL
    try:
        encoded = base64.urlsafe_b64encode(mermaid.encode('utf-8')).decode('ascii').rstrip('=')
        svg_url = f"https://mermaid.ink/svg/{encoded}"
    except Exception as e:
        raise RuntimeError(f"Failed to encode mermaid diagram: {e}") from e

    # Try GET first (fast / cacheable); if that fails, try POST as a fallback
    try:
        response = requests.get(svg_url, timeout=30)
    except requests.RequestException as e:
        response = None
        get_err = e
    else:
        get_err = None

    if response is not None and response.status_code == 200:
        display(SVG(response.content))
        return svg_url

    # GET failed or returned non-200. Try POST fallback and include diagnostic info.
    try:
        post_resp = requests.post("https://mermaid.ink/svg", json={"code": mermaid}, timeout=30)
    except requests.RequestException as e:
        raise Exception(f"GET to {svg_url} failed: {get_err}. POST fallback also failed: {e}") from e

    if post_resp.status_code == 200:
        display(SVG(post_resp.content))
        return "POST fallback (SVG returned)"

    # Both failed — raise detailed error including status codes and truncated bodies for debugging
    get_status = response.status_code if response is not None else 'no-response'
    get_text = (response.text[:1000] + '...') if (response is not None and response.text) else ''
    post_text = (post_resp.text[:1000] + '...') if post_resp and post_resp.text else ''
    raise Exception(
        f"Failed to get SVG. GET {svg_url} -> {get_status}. GET body: {get_text}\nPOST -> {post_resp.status_code}. POST body: {post_text}"
    )

# --- Download Mechanism ---

def download_file(filename, url):
    """Download a file from a URL to the current working directory."""
    if os.path.exists(filename):
        print(f"✅ {filename} already exists locally.")
        return
    
    try:
        print(f"⬇️ Downloading {filename} from {url}...")
        response = requests.get(url)
        response.raise_for_status()
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"✅ {filename} downloaded successfully!")
    except Exception as e:
        print(f"❌ Failed to download {filename}: {e}")

# Base URL for the raw files on GitHub
base_url = "https://raw.githubusercontent.com/StephenEastham/bmw-sales-forecast/refs/heads/main/project-documents/py/mermaid_scripts"

# List of files to download and render
files_to_process = [
    "alert_state_machine.md",
    "data_schema.mmd",
    "forecast_algo.mmd",
    "main_detailed.mmd",
    "reporting_flow.md"
]

print("--- 1. Downloading Files from GitHub ---")
for filename in files_to_process:
    url = f"{base_url}/{filename}"
    download_file(filename, url)
print("\n")

print("--- 2. Rendering Downloaded Files (Relative Paths) ---")
for filename in files_to_process:
    print(f"Rendering {filename}...")
    try:
        mm_svg(filename, is_path=True)
    except Exception as e:
        print(f"Error rendering {filename}: {e}")
print("\n")

""" print("--- 3. Rendering Local Absolute Paths (Original Request) ---")
local_paths = [
    r"C:\Users\easts\github\bmw-sales-forecast\project-documents\py\mermaid_scripts\alert_state_machine.md",
    r"C:\Users\easts\github\bmw-sales-forecast\project-documents\py\mermaid_scripts\data_schema.mmd",
    r"C:\Users\easts\github\bmw-sales-forecast\project-documents\py\mermaid_scripts\forecast_algo.mmd",
    r"C:\Users\easts\github\bmw-sales-forecast\project-documents\py\mermaid_scripts\main_detailed.mmd",
    r"C:\Users\easts\github\bmw-sales-forecast\project-documents\py\mermaid_scripts\reporting_flow.md"
]

for path in local_paths:
    print(f"Rendering {path}...")
    try:
        mm_svg(path, is_path=True)
    except Exception as e:
        print(f"Error rendering {path}: {e}") """